# Question 1: LLM Integration

In [ ]:
pip install google-generativeai python-dotenv


In [84]:
import google.generativeai as genai
import os
import re
from dotenv import load_dotenv, find_dotenv


load_dotenv(find_dotenv())

def get_api_key(key: str) -> str | None:
    return os.getenv(key)


In [85]:
OPENAI_API_KEY = get_api_key("OPEN_AI_API_KEY")
GEMINI_API_KEY = get_api_key("GEMINI_API_KEY")


In [63]:
import nest_asyncio
nest_asyncio.apply()

## 1.1 Single Text Translation: 
You are asked to write a Python code using the OpenAI API to translate a given text into
Vietnamese. (You should check the text if it’s already the destination language).


For example, translating "Hello" into Vietnamese should return "Xin chào", but “Xin chào”
should return the same.

In [64]:
def is_vietnamese(text):
    vietnamese_chars = (
    r'[àáảãạăằắẳẵặâầấẩẫậđèéẻẽẹêềếểễệìíỉĩị'
    r'òóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵ]'
    )

    return bool(re.search(vietnamese_chars, text))

In [65]:
import google.generativeai as genai
genai.configure(api_key = GEMINI_API_KEY)

model = genai.GenerativeModel('gemini-1.5-flash')

def translate_single_gemini(json_input):
    text = json_input['text']
    dest_language = json_input['dest_language']
    
    # Nếu văn bản đã là tiếng Việt, trả về luôn
    if is_vietnamese(text):
        return text
    try:
        # Tạo prompt dịch thuật
        prompt = f"Translate the following text into {dest_language} language: '{text}'"

        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        print(f"Error during translation with Gemini: {e}")
    return None

## 1.2 Multiple Texts Translation: 
Similar to 1.1, but the input is a list of texts. The Python code should accept a list of strings and
return their translations in the specified language. For instance, translating ["Hello", "I am
John", “Tôi là sinh viên”] into Vietnamese should return ["Xin chào", "Tôi tên là John", “Tôi là
sinh viên”]

In [66]:
def translate_multiple_gemini(json_input):
    texts = json_input['text']
    dest_language = json_input['dest_language']

    # Đảm bảo đầu vào là danh sách
    if isinstance(texts, str):
        texts = [texts]
        
    translations = []
    for text in texts:
        try:
            # Tạo JSON đầu vào cho từng văn bản
            single_json = {"text": text, "dest_language": dest_language}
            translation = translate_single_gemini(single_json)
            translations.append(translation)
        except Exception as e:
            print(f"Error translating text '{text}': {e}")
            translations.append(None)
    return translations

In [67]:
def main():
    json_1 = {
        "text": "Hello",
        "dest_language": "Vietnamese"
    }
    json_2 = {
        "text": "Xin chào",
        "dest_language": "Vietnamese"
    }

    json_3 = {
        "text": "Tôi tên Nam",
        "dest_language": "Vietnamese"
    }
    json_4 = {
        "text": ["Hello", "I am John", "Tôi là sinh viên"],
        "dest_language": "Vietnamese"
    }
    # Kiểm tra hàm
    print(translate_single_gemini(json_1))
    print(translate_single_gemini(json_2))
    print(translate_single_gemini(json_3))
    print(translate_multiple_gemini(json_4))

In [68]:
if __name__ == '__main__':
    main()

Xin chào
Xin chào
Tôi tên Nam
['Xin chào', 'Tôi là John.', 'Tôi là sinh viên']


# Question 2: Chatbot Development
Assignment Test: Chatbot Development from Website Data. The data is at
https://www.presight.io/privacy-policy.html

## 2.1 Data Access and Indexing 
Tasked with creating a chatbot, begin by web crawling the specified website to gather relevant
data, then preprocess and structure this data into a searchable index, ready for query retrieval.
(Short version: crawling then embedding data, you can use selenium or
requests)

In [ ]:
pip install selenium beautifulsoup4 requests webdriver-manager


In [171]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service 
from selenium.common.exceptions import NoSuchElementException
import requests
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [15]:
main_link = 'https://www.presight.io/privacy-policy.html'

chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument('--disable-infobars')

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get(main_link)

In [16]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [17]:
data = []

In [18]:
title = soup.title.get_text()
print(title)
data.append(title)

Presight | AI-powered data platform for business teams


In [19]:
head_above = soup.find("div", {'class': "chakra-stack css-1uji4px"}) \
                .find("h2", {'class': "chakra-heading css-dhb2ck"})
if head_above:
    head_above_text = head_above.get_text()
    print(head_above_text)
    data.append(head_above_text)

PRIVACY POLICY


In [20]:
head_below = soup.find("div", {'class': "chakra-stack css-1uji4px"}) \
 .find("h2", {'class': "chakra-heading css-18j379d"})
if head_below:
    head_below_text = head_below.get_text()
    print(head_below_text)
    data.append(head_below_text)

Last updated 15 Sep 2023


In [21]:
content_head = soup.find("div", {'class': "chakra-stack css-lcbvq9"}) \
 .find("p", {"class": "chakra-text css-0"})
if content_head:
    content_head_text = content_head.get_text()
    print(content_head_text)
    data.append(content_head_text)

At Presight, we are committed to protecting the privacy of our customers and visitors to our website. This Privacy Policy explains how we collect, use, and disclose information about our customers and visitors.


In [22]:
current_01 = soup.find("div", {'class': "chakra-stack css-o5l3sd"})
heading = current_01.find("h2", {'class': "chakra-heading css-18j379d"})    

heading_text = heading.get_text(strip=True)
print(heading_text)
data.append(heading_text)
p = heading.find_next("p")
p_text = p.get_text(strip=True)
print(p_text)
data.append(p_text)

Information Collection and Use
We collect several different types of information for various purposes to provide and improve our Service to you.


In [23]:
current_02 = current_01.find_next_sibling("div",{'class':"chakra-stack css-o5l3sd"})

heading = current_02.find("h2", {'class': "chakra-heading css-18j379d"})
heading_text = heading.get_text(strip=True)
print(heading_text)
data.append(heading_text)

tag = current_02.find("div", {'class': "chakra-stack css-bel3sh"})
child_01 = tag.find("div", {'class': "css-0"})
if child_01:
    i = child_01.find("i")
    i_text = i.get_text(strip=True)
    print(i_text)
    data.append(i_text)
    p = child_01.find_next("p")
    p_text = p.get_text(strip=True)
    print(p_text)
    data.append(p_text)

    ul = child_01.find("ul")
    for item in ul.find_all("li"):
        item_text = item.get_text(strip=True)
        print(item_text)
        data.append(item_text)


child_02 = child_01.find_next_sibling("div", {'class': "css-0"})
if child_02:
    i = child_02.find("i")
    i_text = i.get_text(strip=True)
    print(i_text)
    data.append(i_text)
    p = child_02.find_next("p")
    p_text = p.get_text(strip=True)
    print(p_text)
    data.append(p_text)

Types of Data Collected
Personal Data
While using our Service, we may ask you to provide us with certain personally identifiable information that can be used to contact or identify you ("Personal Data"). Personally identifiable information may include, but is not limited to:
Email address
First name and last name
Phone number
Address, State, Province, ZIP/Postal code, City
Cookies and Usage Data
Usage Data
We may also collect information that your browser sends whenever you visit our Service or when you access the Service by or through a mobile device ("Usage Data"). This Usage Data may include information such as your computer's Internet Protocol address (e.g. IP address), browser type, browser version, the pages of our Service that you visit, the time and date of your visit, the time spent on those pages, unique device identifiers, and other diagnostic data.


In [24]:
current_03 = current_02.find_next_sibling("div",{'class':"chakra-stack css-o5l3sd"})

heading = current_03.find("h2", {'class': "chakra-heading css-18j379d"})
heading_text = heading.get_text(strip=True)
print(heading_text)
data.append(heading_text)

p = current_03.find_next("p")
p_text = p.get_text(strip=True)
print(p_text)
data.append(p_text)
ul = current_03.find("ul")
for item in ul.find_all("li"):
    item_text = item.get_text(strip=True)
    print(item_text)
    data.append(item_text)

Use of Data
Presight uses the collected data for various purposes:
To provide and maintain our Service
To notify you about changes to our Service
To allow you to participate in interactive features of our Service when you choose to do so
To provide customer support
To gather analysis or valuable information so that we can improve our Service
To monitor the usage of our Service
To detect, prevent and address technical issues


In [25]:
current_04 = current_03.find_next_sibling("div",{'class':"chakra-stack css-o5l3sd"})

heading = current_04.find("h2", {'class': "chakra-heading css-18j379d"})
heading_text = heading.get_text(strip=True)
print(heading_text)
data.append(heading_text)

p = current_04.find_next("p")
p_text = p.get_text(strip=True)
print(p_text)
data.append(p_text)

Consent
As personal information is collected, you will be asked to confirm that your information is correct prior to submitting it to Presight.


In [26]:
current_05 = current_04.find_next_sibling("div",{'class':"chakra-stack css-o5l3sd"})

heading = current_05.find("h2", {'class': "chakra-heading css-18j379d"})
heading_text = heading.get_text(strip=True)
print(heading_text)
data.append(heading_text)

h2_01 = heading.find_next('h2')
h2_01_text = h2_01.get_text(strip=True)
print(h2_01_text)
data.append(h2_01_text)

p_01 = heading.find_next('p')
p_01_text = p_01.get_text(strip=True)
print(p_01_text)
data.append(p_01_text)

h2_02 = h2_01.find_next('h2')
h2_02_text = h2_02.get_text(strip=True)
print(h2_02_text)
data.append(h2_02_text)

p_02 = p_01.find_next('p')
p_02_text = p_02.get_text(strip=True)
print(p_02_text)
data.append(p_02_text)

Access to Personal Information
Accessing Your Personal Information
You have the right to access all of your personal information that we hold. Through the application, you can correct, amend, or append your personal information by logging into the application and navigating to your settings and profile.
Automated Edit Checks
Presight employs automated edit checks to ensure that data entry fields are completed properly when collecting personal information. These edit checks help maintain data integrity and accuracy. You are encouraged to provide complete and valid information to ensure the smooth processing of their personal data.


In [27]:
current_06 = current_05.find_next_sibling("div",{'class':"chakra-stack css-o5l3sd"})

heading = current_06.find("h2", {'class': "chakra-heading css-18j379d"})
heading_text = heading.get_text(strip=True)
print(heading_text)
data.append(heading_text)

p = current_06.find_next("p")
p_text = p.get_text(strip=True)
print(p_text)
data.append(p_text)

Disclosure of Information
We may disclose your application data to third-party service providers who help us provide our services such as Datadog, AWS, Google Cloud and Google Workspace. We may also disclose your information in response to a legal request, such as a subpoena or court order, or to protect our rights or the rights of others.


In [28]:
current_07 = current_06.find_next_sibling("div",{'class':"chakra-stack css-o5l3sd"})

heading = current_07.find("h2", {'class': "chakra-heading css-18j379d"})
heading_text = heading.get_text(strip=True)
print(heading_text)
data.append(heading_text)

p = current_07.find_next("p")
p_text = p.get_text(strip=True)
print(p_text)
data.append(p_text)

Sharing of Personal Data
Your personal data will not be subject to sharing, transfer, rental or exchange for the benefit of third parties, including AI models.


In [30]:
current_08 = current_07.find_next_sibling("div",{'class':"chakra-stack css-o5l3sd"})

heading = current_08.find("h2", {'class': "chakra-heading css-18j379d"})
heading_text = heading.get_text(strip=True)
print(heading_text)
data.append(heading_text)

p = current_08.find_next("p")
p_text = p.get_text(strip=True)
print(p_text)
data.append(p_text)

ul = current_08.find("ul")
for item in ul.find_all("li"):
 item_text = item.get_text(strip=True)
 print(item_text)
 data.append(item_text)

Google User Data and Google Workspace APIs
In all cases when users authenticate the platform to Google Workspace, the following applies:
We do not retain or use Google User Data to develop, improve, or train generalized/non-personalized AI and/or ML models.
We do not use Google Workspace APIs to develop, improve, or train generalized/non-personalized AI and/or ML models.
We do not transfer Google User Data to third-party AI tools for the purpose of developing, improving, or training generalized or non-personalized AI and/or ML models.


In [31]:
current_09 = current_08.find_next_sibling("div",{'class':"chakra-stack css-o5l3sd"})

heading = current_09.find("h2", {'class': "chakra-heading css-18j379d"})
heading_text = heading.get_text(strip=True)
print(heading_text)
data.append(heading_text)

ul = current_09.find("ul")
for item in ul.find_all("li"):
    item_text = item.get_text(strip=True)
    print(item_text)
    data.append(item_text)

Data Security
All data is encrypted both in transit and at rest, using industry-standard encryption methods.
We regularly perform security audits and vulnerability assessments to ensure the safety of our platform and the data stored within it.
Our employees are trained on best practices for data security, and access to customer data is restricted on a need-to-know basis.


In [32]:
current_10 = current_09.find_next_sibling("div",{'class':"chakra-stack css-o5l3sd"})

heading = current_10.find("h2", {'class': "chakra-heading css-18j379d"})
heading_text = heading.get_text(strip=True)
print(heading_text)
data.append(heading_text)

p = current_10.find_next("p")
p_text = p.get_text(strip=True)
print(p_text)
data.append(p_text)

Data Retention & Disposal
Customer data is retained for as long as the account is in active status. Data enters an “expired” state when the account is voluntarily closed. Expired account data will be retained for 60 days. After this period, the account and related data will be removed.


In [33]:
current_11 = current_10.find_next_sibling("div",{'class':"chakra-stack css-o5l3sd"})

heading = current_11.find("h2", {'class': "chakra-heading css-18j379d"})
heading_text = heading.get_text(strip=True)
print(heading_text)
data.append(heading_text)
ul = current_11.find("ul")

for item in ul.find_all("li"):
    item_text = item.get_text(strip=True)
    print(item_text)
    data.append(item_text)

Quality, Including Data Subjects' Responsibilities for Quality
We are committed to maintaining the quality and accuracy of the personal information we collect and process.
We rely on data subjects to provide accurate and up-to-date information.
Data subjects have the responsibility to inform us of any changes or inaccuracies in their personal data.
If you believe that any information we hold about you is inaccurate, incomplete, or outdated, please contact us promptly to rectify the information.


In [34]:
current_12 = current_11.find_next_sibling("div",{'class':"chakra-stack css-o5l3sd"})
heading = current_12.find("h2", {'class': "chakra-heading css-18j379d"})
heading_text = heading.get_text(strip=True)
print(heading_text)
data.append(heading_text)
ul = current_12.find("ul")
for item in ul.find_all("li"):
    item_text = item.get_text(strip=True)
    print(item_text)
    data.append(item_text)

Monitoring and Enforcement
We regularly monitor its data processing activities to ensure compliance with this privacy policy and applicable data protection laws.
In the event of a data breach or any unauthorized access to your personal information, we will notify you and the appropriate authorities as required by law.
We committed to cooperating with data protection authorities and complying with their advice and decisions regarding data protection and privacy matters.


In [35]:
current_13 = current_12.find_next_sibling("div",{'class':"chakra-stack css-o5l3sd"})

heading = current_13.find("h2", {'class': "chakra-heading css-18j379d"})
heading_text = heading.get_text(strip=True)
print(heading_text)
data.append(heading_text)

p = current_13.find_next("p")
p_text = p.get_text(strip=True)
print(p_text)
data.append(p_text)

Cookies
We use cookies to enhance your experience on our website. You can control the use of cookies through your web browser settings.


In [36]:
current_14 = current_13.find_next_sibling("div",{'class':"chakra-stack css-o5l3sd"})

heading = current_14.find("h2", {'class': "chakra-heading css-18j379d"})
heading_text = heading.get_text(strip=True)
print(heading_text)
data.append(heading_text)

p = current_14.find_next("p")
p_text = p.get_text(strip=True)
print(p_text)
data.append(p_text) 

Third-Party Websites
Our website may contain links to third-party websites. We are not responsible for the privacy practices or content of those websites.


In [37]:
current_15 = current_14.find_next_sibling("div",{'class':"chakra-stack css-o5l3sd"})

heading = current_15.find("h2", {'class': "chakra-heading css-18j379d"})
heading_text = heading.get_text(strip=True)
print(heading_text)
data.append(heading_text)

p = current_15.find_next("p")
p_text = p.get_text(strip=True)
print(p_text)
data.append(p_text)

Changes to Privacy Policy
We may update this Privacy Policy from time to time. The updated Privacy Policy will be posted on our website.


In [38]:
current_16 = current_15.find_next_sibling("div",{'class':"chakra-stack css-o5l3sd"})
heading = current_16.find("h2", {'class': "chakra-heading css-18j379d"})
heading_text = heading.get_text(strip=True)
print(heading_text)

data.append(heading_text)
p = current_16.find_next("p")
p_text = p.get_text(strip=True)
print(p_text)
data.append(p_text)

Contact Us
If you have any questions about this Privacy Policy, please contact us through the customer portal or by email atpresight@presight.io.


In [39]:
current_17 = current_16.find_next_sibling("div",{'class':"chakra-stack css-o5l3sd"})
heading = current_17.find("h2", {'class': "chakra-heading css-18j379d"})
heading_text = heading.get_text(strip=True)
print(heading_text)
data.append(heading_text)
p = current_17.find_next("p")
p_text = p.get_text(strip=True)
print(p_text)
data.append(p_text)

Purposeful Use Only
We commit to only use personal information for the purposes identified in the entity's privacy policy.


In [ ]:
pip install numpy pandas

In [42]:
import pandas as pd
import numpy as np
df = pd.DataFrame(data, columns=["Content"])
df.head()

,Content
0,Presight | AI-powered data platform for busine...
1,PRIVACY POLICY
2,Last updated 15 Sep 2023
3,"At Presight, we are committed to protecting th..."
4,Information Collection and Use


In [43]:
df.to_csv('dataset.csv', index=False, encoding='utf-8')

## 2.2 Chatbot Development 
Develop a chatbot that employs natural language processing to comprehend user questions,
searches the indexed data from 2.1 for the best match, and delivers the most accurate
response drawn from the website's information. (Use any distance/similarity metrics
to get the best match paragraph then feed to LLM to get answer)

In [ ]:
pip install llama-index


In [86]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import SummaryIndex, VectorStoreIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector
from llama_index.core.vector_stores import FilterCondition
from llama_index.core.vector_stores import MetadataFilters
from llama_index.core.tools import FunctionTool
import time
from typing import List
llm = OpenAI(model="gpt-3.5-turbo", api_key=OPENAI_API_KEY)
embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

### Define Router Query Engine

In [87]:
def get_router_query_engine(file_path: str, llm = None, embed_model = None):
    """Get router query engine."""
    # Bắt đầu đo thời gian
    start_time = time.time()

    # Load documents
    documents = SimpleDirectoryReader(input_files=[file_path]).load_data()

    splitter = SentenceSplitter(chunk_size=512)
    nodes = splitter.get_nodes_from_documents(documents)

    summary_index = SummaryIndex(nodes, llm=llm)
    # SummaryIndex sử dụng mô hình ngôn ngữ để đọc qua nội dung của các nodes,
    # tạo ra các bản tóm tắt.
    # Khi có một truy vấn, nó sẽ dựa vào thông tin tóm tắt để trả lời câu hỏi
    # hoặc cung cấp nội dung cô đọng
    vector_index = VectorStoreIndex(nodes, embed_model=embed_model)
    # Sử dụng một mô hình nhúng để chuyển các nodes thành vector số.
    # Khi nhận được một truy vấn, nó cũng được chuyển thành vector số.
    # Sau đó, quá trình tìm kiếm sẽ dựa trên độ tương đồng giữa các vector.

    # Khởi tạo các Query Engines (công cụ truy vấn) từ
    # hai đối tượng summary_index và vector_index
    summary_query_engine = summary_index.as_query_engine(
            response_mode="tree_summarize",
            use_async=True,
            llm=llm
            ) # được dùng để xây dựng công cụ truy vấn cho mục đích tóm tắt.
    vector_query_engine = vector_index.as_query_engine(llm=llm)
    # công cụ truy vấn liên quan đến xử lý
    # hoặc tìm kiếm dữ liệu theo không gian vector

    summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
    "Useful for summarization questions related to Presight web"
    ),
    )
    vector_tool = QueryEngineTool.from_defaults(
        query_engine=vector_query_engine,
        description=(
        "Useful for retrieving specific context from the Presight web."
        ),
    )

    # Tạo công cụ truy vấn có thể định tuyến các truy vấn đến
    # các "Query Engine Tool" phù hợp
    query_engine = RouterQueryEngine(
        selector=LLMSingleSelector.from_defaults(),
        query_engine_tools=[
        summary_tool,
        vector_tool,
        ],
        verbose=True
        )
    # Đo thời gian kết thúc
    end_time = time.time()
    total_time = end_time - start_time
    return query_engine, total_time

In [88]:
query_engine, total_time = get_router_query_engine("dataset.csv", llm=llm, embed_model=embed_model)

response = query_engine.query("summary the presight web")
print(f"Answer: {str(response)}")
print(f"Running time to initialize the query engine: {total_time:.2f} seconds")

Selecting query engine 0: Useful for summarization questions related to Presight web.
Answer: Presight is an AI-powered data platform that is committed to protecting the privacy of its customers and website visitors. They collect various types of information, including personal data like email addresses, names, phone numbers, and addresses, as well as usage data such as IP addresses and browser information. The collected data is used for purposes like maintaining the service, providing customer support, and improving the platform. Presight ensures data security through encryption methods, regular security audits, and employee training. They do not share personal data with third parties, and they have measures in place to allow users to access and correct their personal information. Additionally, they use cookies on their website and may update their privacy policy periodically.
Running time to initialize the query engine: 3.23 seconds


In [77]:
response = query_engine.query("What is Policy ?")
print(f"Answer: {str(response)}")
print(f"Running time to initialize the query engine: {total_time:.2f} seconds")

Selecting query engine 1: Policy is more about retrieving specific context rather than summarization.
Answer: The policy outlined in the context information pertains to privacy policy, data security, data retention and disposal, quality of data, and monitoring and enforcement practices related to data protection and privacy matters.
Running time to initialize the query engine: 4.24 seconds


### Tool calling 
Define the Auto-Retrival Tool

In [78]:
llm = OpenAI(model="gpt-3.5-turbo", api_key= OPENAI_API_KEY)
documents = SimpleDirectoryReader(input_files=["dataset.csv"]).load_data()

splitter = SentenceSplitter(chunk_size=512)
nodes = splitter.get_nodes_from_documents(documents)

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

In [79]:
def vector_query(
 query: str,
 page_numbers: List[str]
) -> str:
    metadata_dicts = [
        {"key": "page_label", "value": p} for p in page_numbers
    ]

    query_engine = vector_index.as_query_engine(
        similarity_top_k=1,
        filters=MetadataFilters.from_dicts(
            metadata_dicts,
            condition=FilterCondition.OR
    )
    )
    response = query_engine.query(query)
    return response

vector_query_tool = FunctionTool.from_defaults(
    name="vector_tool",
    fn=vector_query,
    description="Useful for retrieving specific context from the Presight web."
)

In [80]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
    )

summary_tool = QueryEngineTool.from_defaults(
    name="summary_tool",
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to Presight web"
    ),
)

In [81]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool],
    "What is PRIVACY POLICY?",
    verbose=True
)

=== Calling Function ===
Calling function: summary_tool with args: {"input": "Privacy Policy"}
=== Function Output ===
Presight is dedicated to safeguarding the privacy of its customers and website visitors. The Privacy Policy outlines the collection, use, and disclosure of information, including personal data like email addresses, names, phone numbers, and addresses. The company uses collected data for various purposes such as maintaining services, notifying about changes, providing customer support, and improving services. Users are asked to confirm the accuracy of their personal information before submission. Access to personal information is granted to users, who can correct or update their details through the application. Presight employs automated edit checks to ensure data accuracy. Data security measures include encryption, regular security audits, and employee training. Customer data is retained while the account is active and for 60 days after closure. The company emphasizes 

In [82]:
response = llm.predict_and_call(
    [vector_query_tool, summary_tool],
    "What is PRIVACY POLICY on page 2?",
    verbose=True
)

=== Calling Function ===
Calling function: vector_tool with args: {"query": "PRIVACY POLICY", "page_numbers": ["2"]}
=== Function Output ===
Encountered error: Vector store query result should return at least one of nodes or ids.
